In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
# Prevent GPU memory crash
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_memory_growth(gpus[0], True)

In [ ]:
base_dir = "/content/drive/MyDrive/fabric-dataset"

train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "validation")
test_dir  = os.path.join(base_dir, "test")

img_size = 128
batch_size = 16
epochs = 40
num_classes = 4


In [ ]:
train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=8,
    zoom_range=0.08,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1./255)
test_gen = ImageDataGenerator(rescale=1./255)


In [ ]:
train_data = train_gen.flow_from_directory(
    train_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    classes=["cotton-fabric", "denim-fabric", "silk-fabric", "nylon-fabric"]
)

val_data = val_gen.flow_from_directory(
    val_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    classes=["cotton-fabric", "denim-fabric", "silk-fabric", "nylon-fabric"]
)

test_data = test_gen.flow_from_directory(
    test_dir,
    target_size=(img_size, img_size),
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    classes=["cotton-fabric", "denim-fabric", "silk-fabric", "nylon-fabric"]
)


Found 2459 images belonging to 4 classes.
Found 750 images belonging to 4 classes.
Found 387 images belonging to 4 classes.


In [ ]:
model = Sequential([
    Conv2D(32, (3,3), activation="relu", input_shape=(img_size, img_size, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation="relu"),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(128, activation="relu"),
    Dense(num_classes, activation="softmax")
])

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,662,596 (6.34 MB)

 Trainable params: 1,662,596 (6.34 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
early_stop = EarlyStopping(
    monitor="val_accuracy",
    patience=5,
    restore_best_weights=True
)


In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=epochs,
    callbacks=[early_stop]
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/40
154/154 ━━━━━━━━━━━━━━━━━━━━ 259s 2s/step - accuracy: 0.6124 - loss: 1.0020 - val_accuracy: 0.7013 - val_loss: 0.8569
Epoch 2/40
154/154 ━━━━━━━━━━━━━━━━━━━━ 119s 770ms/step - accuracy: 0.7436 - loss: 0.7121 - val_accuracy: 0.6080 - val_loss: 0.9067
Epoch 3/40
154/154 ━━━━━━━━━━━━━━━━━━━━ 117s 759ms/step - accuracy: 0.7715 - loss: 0.6259 - val_accuracy: 0.7293 - val_loss: 0.6746
Epoch 4/40
154/154 ━━━━━━━━━━━━━━━━━━━━ 117s 757ms/step - accuracy: 0.7800 - loss: 0.5849 - val_accuracy: 0.6867 - val_loss: 0.8898
Epoch 5/40
154/154 ━━━━━━━━━━━━━━━━━━━━ 147s 788ms/step - accuracy: 0.8013 - loss: 0.5395 - val_accuracy: 0.6640 - val_loss: 0.9768
Epoch 6/40
154/154 ━━━━━━━━━━━━━━━━━━━━ 116s 750ms/step - accuracy: 0.8293 - loss: 0.4918 - val_accuracy: 0.7907 - val_loss: 0.5233
Epoch 7/40
154/154 ━━━━━━━━━━━━━━━━━━━━ 116s 752ms/step - accuracy: 0.8405 - loss: 0.4219 - val_accuracy: 0.7773 - val_loss: 0.6363
Epoch 8/40
154/154 ━━━━━━━━━━━━━━━━━━━━ 117s 756ms/step - accuracy: 0.8466 - lo

In [ ]:
loss, accuracy = model.evaluate(test_data)
print(f"Test Accuracy: {accuracy*100:.2f}%")

25/25 ━━━━━━━━━━━━━━━━━━━━ 95s 4s/step - accuracy: 0.8611 - loss: 0.5177
Test Accuracy: 89.15%


In [ ]:
predictions = model.predict(test_data)
y_pred = np.argmax(predictions, axis=1)
y_true = test_data.classes

print("Confusion Matrix:")
print(confusion_matrix(y_true, y_pred))

print("\nClassification Report:")
print(classification_report(
    y_true, y_pred,
    target_names=["cotton", "denim", "silk", "nylon"]
))


25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 295ms/step
Confusion Matrix:
[[207   8  13   8]
 [  2  64   0   0]
 [  0   0  19   1]
 [  4   0   6  55]]

Classification Report:
              precision    recall  f1-score   support

      cotton       0.97      0.88      0.92       236
       denim       0.89      0.97      0.93        66
        silk       0.50      0.95      0.66        20
       nylon       0.86      0.85      0.85        65

    accuracy                           0.89       387
   macro avg       0.81      0.91      0.84       387
weighted avg       0.91      0.89      0.90       387



In [ ]:
model.save("/content/drive/MyDrive/fabric_detection_model.h5")
print("Model saved successfully")

Model saved successfully
